# PLATEAU データ処理ノートブック

このノートブックでは、PLATEAUの3D都市モデルデータを取得・処理し、水没シミュレーションで使用できる形式に変換します。

## 1. 必要なライブラリのインポート

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
import json
import requests
import zipfile
import io
from tqdm.notebook import tqdm
import warnings
import folium
warnings.filterwarnings('ignore')

print("✅ ライブラリのインポート完了")

ModuleNotFoundError: No module named 'geopandas'

## 2. データディレクトリの設定

In [ ]:
# プロジェクトのルートディレクトリ
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data" / "plateau"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

# ディレクトリ作成
DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 データディレクトリ: {DATA_DIR}")
print(f"📁 出力ディレクトリ: {OUTPUT_DIR}")

## 3. 既存のGMLファイルの確認

In [ ]:
# GMLファイルを検索
gml_files = list(DATA_DIR.glob("*_bldg_*.gml"))
geojson_files = list(DATA_DIR.glob("*.geojson"))

print(f"📊 データファイル統計:")
print(f"  GMLファイル: {len(gml_files)}個")
print(f"  GeoJSONファイル: {len(geojson_files)}個")

if gml_files:
    print(f"\n最初の5個のGMLファイル:")
    for f in gml_files[:5]:
        print(f"  - {f.name}")

if geojson_files:
    print(f"\nGeoJSONファイル:")
    for f in geojson_files:
        print(f"  - {f.name}")

## 4. GMLファイルの処理（CityGML → GeoJSON変換）

In [ ]:
def process_gml_files(gml_files, max_files=10, sample_rate=1.0):
    """
    GMLファイルを処理してGeoDataFrameに変換
    
    Parameters:
    -----------
    gml_files : list
        処理するGMLファイルのリスト
    max_files : int
        処理する最大ファイル数
    sample_rate : float
        各ファイルからサンプリングする割合（0.0-1.0）
    """
    all_buildings = []
    processed_files = 0
    
    # 処理するファイル数を制限
    files_to_process = gml_files[:min(max_files, len(gml_files))]
    
    print(f"🔄 {len(files_to_process)}個のGMLファイルを処理します...")
    
    for gml_file in tqdm(files_to_process, desc="GML処理中"):
        try:
            # GMLファイルを読み込み
            gdf = gpd.read_file(gml_file, driver='GML')
            
            if len(gdf) == 0:
                continue
            
            # サンプリング
            if sample_rate < 1.0:
                gdf = gdf.sample(frac=sample_rate)
            
            # 建物データを抽出
            for idx, row in gdf.iterrows():
                if row.geometry is None:
                    continue
                
                building = {
                    'geometry': row.geometry,
                    'source_file': gml_file.name
                }
                
                # 高さ情報を取得
                height_columns = ['measuredHeight', 'bldg:measuredHeight', 'height', 'Height']
                height = 20  # デフォルト値
                
                for col in height_columns:
                    if col in row and pd.notna(row[col]):
                        try:
                            height = float(row[col])
                            break
                        except:
                            pass
                
                building['height'] = height
                building['ground_height'] = 0
                
                # その他の属性
                if 'usage' in row:
                    building['usage'] = str(row['usage'])
                
                all_buildings.append(building)
            
            processed_files += 1
            
        except Exception as e:
            print(f"\n⚠️ エラー ({gml_file.name}): {str(e)[:100]}")
            continue
    
    print(f"\n✅ {processed_files}個のファイルから{len(all_buildings)}件の建物を抽出")
    
    if all_buildings:
        return gpd.GeoDataFrame(all_buildings)
    else:
        return None

In [ ]:
# GMLファイルを処理
if gml_files:
    buildings_gdf = process_gml_files(gml_files, max_files=10, sample_rate=0.5)
    
    if buildings_gdf is not None:
        # 座標系を設定
        buildings_gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)
        
        # 統計情報
        print(f"\n📊 建物データ統計:")
        print(f"  総建物数: {len(buildings_gdf):,}")
        print(f"  平均高さ: {buildings_gdf['height'].mean():.1f}m")
        print(f"  最高: {buildings_gdf['height'].max():.1f}m")
        print(f"  最低: {buildings_gdf['height'].min():.1f}m")
        
        # GeoJSONとして保存
        output_file = DATA_DIR / "tokyo_bldg.geojson"
        buildings_gdf.to_file(output_file, driver='GeoJSON')
        print(f"\n💾 保存完了: {output_file}")
else:
    print("⚠️ GMLファイルが見つかりません")
    buildings_gdf = None

## 5. 代替データの生成（GMLがない場合）

In [ ]:
def create_sample_tokyo_data():
    """東京の主要エリアのサンプルデータを生成"""
    
    # 東京の主要エリア
    areas = {
        "東京駅": {"center": [139.7671, 35.6812], "buildings": 50, "avg_height": 150},
        "新宿": {"center": [139.7003, 35.6938], "buildings": 60, "avg_height": 180},
        "渋谷": {"center": [139.7019, 35.6580], "buildings": 50, "avg_height": 120},
        "品川": {"center": [139.7388, 35.6284], "buildings": 40, "avg_height": 100},
        "お台場": {"center": [139.7744, 35.6311], "buildings": 30, "avg_height": 60},
    }
    
    from shapely.geometry import Polygon
    
    all_buildings = []
    
    for area_name, info in areas.items():
        lon, lat = info["center"]
        
        for i in range(info["buildings"]):
            # ランダムな位置
            offset_lon = np.random.normal(0, 0.002)
            offset_lat = np.random.normal(0, 0.002)
            
            # 建物の位置
            bldg_lon = lon + offset_lon
            bldg_lat = lat + offset_lat
            
            # 建物のサイズ
            size = 0.0001 * np.random.uniform(0.5, 1.5)
            
            # ポリゴン作成
            poly = Polygon([
                (bldg_lon - size/2, bldg_lat - size/2),
                (bldg_lon + size/2, bldg_lat - size/2),
                (bldg_lon + size/2, bldg_lat + size/2),
                (bldg_lon - size/2, bldg_lat + size/2)
            ])
            
            # 高さ
            height = np.random.normal(info["avg_height"], info["avg_height"] * 0.3)
            height = max(10, min(250, height))  # 10-250mに制限
            
            all_buildings.append({
                'geometry': poly,
                'height': height,
                'ground_height': np.random.uniform(0, 5),
                'area': area_name
            })
    
    return gpd.GeoDataFrame(all_buildings, crs='EPSG:4326')

# データがない場合はサンプルデータを生成
if buildings_gdf is None:
    print("📍 サンプルデータを生成します...")
    buildings_gdf = create_sample_tokyo_data()
    
    # 保存
    output_file = DATA_DIR / "tokyo_bldg.geojson"
    buildings_gdf.to_file(output_file, driver='GeoJSON')
    print(f"✅ サンプルデータを保存: {output_file}")
    print(f"   建物数: {len(buildings_gdf)}")

## 6. データの可視化

In [ ]:
# 建物の分布を地図上に表示
if buildings_gdf is not None and len(buildings_gdf) > 0:
    # 中心座標を計算
    bounds = buildings_gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Foliumマップを作成
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)
    
    # 建物を追加（最大100個まで）
    for idx, building in buildings_gdf.head(100).iterrows():
        # ポリゴンの中心を計算
        centroid = building.geometry.centroid
        
        # 高さによって色を変える
        if building['height'] > 100:
            color = 'red'
        elif building['height'] > 50:
            color = 'orange'
        else:
            color = 'green'
        
        # マーカーを追加
        folium.CircleMarker(
            location=[centroid.y, centroid.x],
            radius=3,
            popup=f"高さ: {building['height']:.1f}m",
            color=color,
            fill=True,
            fillColor=color
        ).add_to(m)
    
    # 地図を表示
    display(m)
    print(f"\n🗺️ {min(100, len(buildings_gdf))}件の建物を地図上に表示")

## 7. 水没シミュレーションのテスト

In [ ]:
# シミュレーション用にパスを追加
import sys
sys.path.append(str(PROJECT_ROOT / 'src'))

try:
    from simulator import FloodSimulator
    from visualizer import FloodVisualizer
    
    # シミュレーション実行
    sim = FloodSimulator("tokyo", data_dir=DATA_DIR)
    sim.load_buildings()
    
    # 水位10mでシミュレーション
    results = sim.simulate_flood(water_level=10.0)
    
    print("\n🌊 シミュレーション結果（水位10m）:")
    print(f"  完全水没: {results['flooded_buildings']}棟 ({results['flooded_percentage']:.1f}%)")
    print(f"  部分水没: {results['partially_flooded']}棟")
    print(f"  影響率: {results['affected_percentage']:.1f}%")
    
except Exception as e:
    print(f"⚠️ シミュレーションエラー: {e}")
    print("src/simulator.pyが必要です")

## 8. 複数水位での影響分析

In [ ]:
# 複数の水位でシミュレーション
water_levels = [2, 5, 10, 15, 20, 30]
results_list = []

if buildings_gdf is not None:
    print("📊 水位別影響分析:")
    print("-" * 50)
    print(f"{'水位(m)':>8} | {'影響建物':>10} | {'完全水没':>10} | {'影響率':>8}")
    print("-" * 50)
    
    for level in water_levels:
        # 簡易シミュレーション
        flood_depth = level - buildings_gdf['ground_height']
        flooded = (flood_depth > buildings_gdf['height']).sum()
        partial = ((flood_depth > 0) & (flood_depth <= buildings_gdf['height'])).sum()
        affected_pct = (flooded + partial) / len(buildings_gdf) * 100
        
        print(f"{level:8.0f} | {flooded + partial:10d} | {flooded:10d} | {affected_pct:7.1f}%")
        
        results_list.append({
            'water_level': level,
            'flooded': flooded,
            'partial': partial,
            'affected_pct': affected_pct
        })
    
    print("-" * 50)

## 9. 結果の保存

In [ ]:
# 処理結果のサマリーを保存
summary = {
    'processed_date': pd.Timestamp.now().isoformat(),
    'data_source': 'PLATEAU GML' if gml_files else 'Sample Data',
    'total_buildings': len(buildings_gdf) if buildings_gdf is not None else 0,
    'simulation_results': results_list
}

# JSONとして保存
summary_file = OUTPUT_DIR / 'plateau_processing_summary.json'
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"📄 サマリーを保存: {summary_file}")
print("\n✅ 処理完了！")
print("\n次のステップ:")
print("1. python src/main.py simulate tokyo --level 10")
print("2. python src/main.py server --port 8001")